In [2]:
#Librerías
import warnings, re
warnings.filterwarnings("ignore")

from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Rutas del proyecto 
PROJECT_ROOT = Path.cwd().parents[0] if Path.cwd().name == 'notebooks' else Path.cwd()
DATA_DIR = PROJECT_ROOT / 'data'
MODELS_DIR = PROJECT_ROOT / 'models'
MODELS_DIR.mkdir(parents=True, exist_ok=True)

# Métrica RMSE compatible
def rmse(y_true, y_pred):
    return mean_squared_error(y_true, y_pred) ** 0.5

DATA_DIR, MODELS_DIR


(WindowsPath('d:/Documentoss/Politécnico. Tecnicatura/2025/2 Cuatri/Aprendizaje Automatico/Trabajo del Parcial/Entrega 3/data'),
 WindowsPath('d:/Documentoss/Politécnico. Tecnicatura/2025/2 Cuatri/Aprendizaje Automatico/Trabajo del Parcial/Entrega 3/models'))

In [3]:
# Leemos el dataset imputado
import pandas as pd
from pathlib import Path

def read_csv_smart(path):
    for enc in ['utf-8', 'latin-1']:
        for sep in [',',';','\t']:
            try:
                df = pd.read_csv(path, encoding=enc, sep=sep)
                if df.shape[1] > 1:
                    return df
            except: 
                pass
    raise ValueError(f"No pude leer bien {path}")

DATA_DIR = Path.cwd().parents[0] / 'data'  # si estás en /notebooks
csv_path = DATA_DIR / "DATA SET IMPUTADO. festivales_imputado.csv"
df_imp = pd.read_csv(csv_path, sep=",", encoding="latin-1")
print("Dimensiones:", df_imp.shape)
df_imp.head(5)




Dimensiones: (3255, 11)


,provincia,departamento,nombre,tematica_principal,tipo_de_gestion,mes_de_realizacion,duracion_dias,cantidad_aprox_de_asistentes,tipo_entrada,ultimo_ano_de_realizacion,mes_num
0,Buenos Aires,San Vicente,DÃ­a Nacional del Payador,Tradicionales,PÃºblico,Julio,3,1000,Gratuita,2022.0,7.0
1,Buenos Aires,Lujan,Festival del Pan Dulce en LujÃ¡n,GastronÃ³mica,PÃºblico,Diciembre,3,2000,Gratuita,2022.0,12.0
2,Buenos Aires,Coronel Pringles,Pringles Cine Fest,ArtÃ­stica,PÃºblico,Noviembre,3,250,Gratuita,2022.0,11.0
3,Buenos Aires,General Alvarado,Fiesta del Kiwi: Mar y Sierras,Productiva,NaN,Noviembre,3,11556,NaN,2022.0,NaN
4,Buenos Aires,Bragado,Suena Rock,ArtÃ­stica,NaN,Febrero,3,3000,Gratuita,2023.0,2.0


In [5]:
# Releemos con 'utf-8' y reparamos texto
df_imp = pd.read_csv(csv_path, sep=",", encoding="utf-8", on_bad_lines='skip')

# Limpiamos espacios y reparamos acentos mal codificados
df_imp = df_imp.apply(lambda x: x.str.strip() if x.dtype == "object" else x)



# Vista general
print("Dimensiones:", df_imp.shape)
df_imp.head(3)


Dimensiones: (3255, 11)


,provincia,departamento,nombre,tematica_principal,tipo_de_gestion,mes_de_realizacion,duracion_dias,cantidad_aprox_de_asistentes,tipo_entrada,ultimo_ano_de_realizacion,mes_num
0,Buenos Aires,San Vicente,Día Nacional del Payador,Tradicionales,Público,Julio,3,1000,Gratuita,2022.0,7.0
1,Buenos Aires,Lujan,Festival del Pan Dulce en Luján,Gastronómica,Público,Diciembre,3,2000,Gratuita,2022.0,12.0
2,Buenos Aires,Coronel Pringles,Pringles Cine Fest,Artística,Público,Noviembre,3,250,Gratuita,2022.0,11.0


In [9]:
df_imp.isna().sum().sort_values(ascending=False)


tipo_entrada                    1504
tipo_de_gestion                 1121
ultimo_ano_de_realizacion       1109
mes_num                          922
tematica_principal               363
provincia                          0
departamento                       0
nombre                             0
mes_de_realizacion                 0
duracion_dias                      0
cantidad_aprox_de_asistentes       0
dtype: int64

In [11]:
# Creamos una copia de trabajo del dataset imputado
df_clean = df_imp.copy()


In [12]:
# --- Normalización de nombres en la columna 'mes_de_realizacion' ---

# Convertimos todo a minúsculas, quitamos espacios y acentos
import unidecode

df_clean['mes_de_realizacion'] = (
    df_clean['mes_de_realizacion']
    .astype(str)
    .str.strip()
    .str.lower()
    .apply(unidecode.unidecode)
)

# Reemplazamos variaciones comunes
reemplazos_meses = {
    'enero': 'Enero',
    'febrero': 'Febrero',
    'marzo': 'Marzo',
    'abril': 'Abril',
    'mayo': 'Mayo',
    'junio': 'Junio',
    'julio': 'Julio',
    'agosto': 'Agosto',
    'septiembre': 'Septiembre',
    'setiembre': 'Septiembre',  # por si hay variantes
    'octubre': 'Octubre',
    'noviembre': 'Noviembre',
    'diciembre': 'Diciembre'
}

df_clean['mes_de_realizacion'] = df_clean['mes_de_realizacion'].replace(reemplazos_meses)

print("Valores únicos de 'mes_de_realizacion' normalizados:")
print(sorted(df_clean['mes_de_realizacion'].unique()))

cat_cols = ['provincia', 'tematica_principal', 'tipo_de_gestion', 'mes_de_realizacion', 'tipo_entrada']
for c in cat_cols:
    print(f"\n---- {c} ----")
    print(df_imp[c].value_counts(dropna=False).head(10))


Valores únicos de 'mes_de_realizacion' normalizados:
['Abril', 'Agosto', 'Diciembre', 'Enero', 'Febrero', 'Julio', 'Junio', 'Marzo', 'Mayo', 'Noviembre', 'Octubre', 'Septiembre']

---- provincia ----
provincia
Córdoba         324
Catamarca       311
Chaco           289
Buenos Aires    282
Jujuy           249
Santa Fe        211
Entre Rios      175
Neuquén         172
La Rioja        162
Salta           157
Name: count, dtype: int64

---- tematica_principal ----
tematica_principal
Religiosa / de Culto        456
Cívico / Histórica          454
Artística                   447
NaN                         363
Tradicionales               353
Productiva                  319
Expresiones del carnaval    315
Otra                        216
Gastronómica                157
Folclórico                  128
Name: count, dtype: int64

---- tipo_de_gestion ----
tipo_de_gestion
Público    1568
NaN        1121
Privado     383
Mixto       183
Name: count, dtype: int64

---- mes_de_realizacion ----
mes_de

In [13]:
orden_meses = [
    'Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio',
    'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre'
]

# Convertimos la columna en tipo "Categorical" con orden definido
df_clean['mes_de_realizacion'] = pd.Categorical(
    df_clean['mes_de_realizacion'],
    categories=orden_meses,
    ordered=True
)


In [14]:

# Reemplazos de nulos en texto
df_clean['tematica_principal'] = df_clean['tematica_principal'].fillna('Sin dato')
df_clean['tipo_de_gestion'] = df_clean['tipo_de_gestion'].fillna('No especifica')
df_clean['tipo_entrada'] = df_clean['tipo_entrada'].fillna('No especifica')


# Eliminamos la columna duplicada de mes_num
if 'mes_num' in df_clean.columns:
    df_clean = df_clean.drop(columns=['mes_num'])
    print("Columna 'mes_num' eliminada.")

# Revisamos si quedaron nulos
df_clean.isna().sum()


Columna 'mes_num' eliminada.


provincia                          0
departamento                       0
nombre                             0
tematica_principal                 0
tipo_de_gestion                    0
mes_de_realizacion                 0
duracion_dias                      0
cantidad_aprox_de_asistentes       0
tipo_entrada                       0
ultimo_ano_de_realizacion       1109
dtype: int64

In [15]:
# Eliminamos columnas no relevantes o redundantes
cols_to_drop = ['ultimo_ano_de_realizacion', 'departamento', 'nombre']

for col in cols_to_drop:
    if col in df_clean.columns:
        df_clean = df_clean.drop(columns=[col])
        print(f"Columna '{col}' eliminada del dataset.")




Columna 'ultimo_ano_de_realizacion' eliminada del dataset.
Columna 'departamento' eliminada del dataset.
Columna 'nombre' eliminada del dataset.


In [16]:
# Columnas categóricas a codificar
cat_cols = ['provincia', 'tematica_principal', 'tipo_de_gestion', 'mes_de_realizacion', 'tipo_entrada']

# One-hot encoding con valores binarios
df_cod = pd.get_dummies(df_clean, columns=cat_cols, drop_first=False, dtype=int)

# Verificación
print("Dataset codificado:", df_cod.shape)
df_cod.head(3)



Dataset codificado: (3255, 57)


,duracion_dias,cantidad_aprox_de_asistentes,provincia_Buenos Aires,provincia_Catamarca,provincia_Chaco,provincia_Chubut,provincia_Ciudad Autónoma de Buenos Aires,provincia_Corrientes,provincia_Córdoba,provincia_Entre Rios,...,mes_de_realizacion_Julio,mes_de_realizacion_Agosto,mes_de_realizacion_Septiembre,mes_de_realizacion_Octubre,mes_de_realizacion_Noviembre,mes_de_realizacion_Diciembre,tipo_entrada_Gratuita,tipo_entrada_Mixta,tipo_entrada_No especifica,tipo_entrada_Paga
0,3,1000,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
1,3,2000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
2,3,250,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0


In [17]:
df_cod.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3255 entries, 0 to 3254
Data columns (total 57 columns):
 #   Column                                            Non-Null Count  Dtype
---  ------                                            --------------  -----
 0   duracion_dias                                     3255 non-null   int64
 1   cantidad_aprox_de_asistentes                      3255 non-null   int64
 2   provincia_Buenos Aires                            3255 non-null   int64
 3   provincia_Catamarca                               3255 non-null   int64
 4   provincia_Chaco                                   3255 non-null   int64
 5   provincia_Chubut                                  3255 non-null   int64
 6   provincia_Ciudad Autónoma de Buenos Aires         3255 non-null   int64
 7   provincia_Corrientes                              3255 non-null   int64
 8   provincia_Córdoba                                 3255 non-null   int64
 9   provincia_Entre Rios                     

In [18]:
from pathlib import Path

# Definimos la ruta donde guardarlo
DATA_DIR = Path.cwd().parents[0] / 'data'

# Nombre del archivo final
file_path = DATA_DIR / "DATA SET CODIFICADO.festivales_codificado_modelo.csv"

# Guardamos el CSV
df_cod.to_csv(file_path, index=False, encoding='utf-8')

print(f" Archivo guardado correctamente en: {file_path}")


 Archivo guardado correctamente en: d:\Documentoss\Politécnico. Tecnicatura\2025\2 Cuatri\Aprendizaje Automatico\Trabajo del Parcial\Entrega 3\data\DATA SET CODIFICADO.festivales_codificado_modelo.csv
